In [1]:
import os, time
from tqdm import tqdm
import pandas as pd
from joblib import Parallel, delayed
from ogb.utils.url import download_url, extract_zip

from molecules.physics import generate_physics_dict_structure
from molecules.parse_sdf import sdf_to_mols

In [2]:
USE_MICRO_DATASET = False # Set to false to use the FULL dataset, otherwise a dummy smaller dataset
ROOT = "."
micro_name = "_micro" if USE_MICRO_DATASET else ""
SDF_URL = f"https://datasets-public-research.s3.us-east-2.amazonaws.com/PCQM4M/pcqm4m-v2_sdf{micro_name}.zip"
SDF_LOCAL_ZIP = f"{ROOT}/pcqm4m-v2_sdf{micro_name}.zip"
SDF_LOCAL = os.path.join(os.path.splitext(SDF_LOCAL_ZIP)[0])
RAW_URL = "http://ogb-data.stanford.edu/data/lsc/pcqm4m-v2.zip"
RAW_CSV = f"{ROOT}/pcqm4m-v2/raw/data.csv.gz"
SKIP_DOWNLOAD = True # Set to false for downloading and unzipping the file. If already downloaded, set to True

In [3]:
if not SKIP_DOWNLOAD:
    # This can take many minutes
    path = download_url(SDF_URL, ROOT)
    extract_zip(path, ROOT)

In [4]:
if not SKIP_DOWNLOAD:
    raw_path = download_url(RAW_URL, ROOT)
    extract_zip(raw_path, ROOT)

In [5]:
def get_files_from_dir(dir, ext):
    found_files = []
    all_dirs = os.walk(dir)
    for root, dirs, files in all_dirs:
        for filename in files:
            if filename.endswith(ext):
                found_files.append(os.path.join(root, filename))
    return found_files


In [6]:
sdf_files = get_files_from_dir(SDF_LOCAL, ".sdf")
time.sleep(1)
print(sdf_files[:10])

['./pcqm4m-v2_sdf/00250000_00259999/252506.sdf', './pcqm4m-v2_sdf/00250000_00259999/253520.sdf', './pcqm4m-v2_sdf/00250000_00259999/253300.sdf', './pcqm4m-v2_sdf/00250000_00259999/259104.sdf', './pcqm4m-v2_sdf/00250000_00259999/256366.sdf', './pcqm4m-v2_sdf/00250000_00259999/254181.sdf', './pcqm4m-v2_sdf/00250000_00259999/256379.sdf', './pcqm4m-v2_sdf/00250000_00259999/252575.sdf', './pcqm4m-v2_sdf/00250000_00259999/255187.sdf', './pcqm4m-v2_sdf/00250000_00259999/257670.sdf']


In [7]:
mols = {}
for file in tqdm(sdf_files):
    try:
        idx = int(os.path.basename(os.path.splitext(file)[0]))
        mols[idx] = sdf_to_mols(file)[0]
    except Exception as e:
        print(f"Error loading file {file}")
        pass

  6%|▌         | 210874/3378606 [01:17<18:56, 2786.49it/s]RDKit WARNING: [20:36:17] WARNING: not removing hydrogen atom without neighbors
[20:36:17] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [20:36:17] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [20:36:17] WARNING: not removing hydrogen atom without neighbors
[20:36:17] WARNING: not removing hydrogen atom without neighbors
[20:36:17] WARNING: not removing hydrogen atom without neighbors
  7%|▋         | 222041/3378606 [01:21<18:59, 2770.51it/s]RDKit WARNING: [20:36:21] Warning: molecule is tagged as 3D, but all Z coords are zero
[20:36:21] Warning: molecule is tagged as 3D, but all Z coords are zero
 24%|██▍       | 826602/3378606 [05:06<14:58, 2840.59it/s]RDKit WARNING: [20:40:05] Warning: molecule is tagged as 3D, but all Z coords are zero
[20:40:05] Warning: molecule is tagged as 3D, but all Z coords are zero
 27%|██▋       | 905188/3378606 [05:35<14:34, 2827.69it/s]RDKit WARNING: 

In [8]:
list_struct = Parallel(n_jobs=-1)(delayed(generate_physics_dict_structure)(mol, idx) for idx, mol in tqdm(mols.items()))
# list_struct = []
# for idx, mol in tqdm(mols.items()):
#     list_struct.append(generate_physics_dict_structure(mol, idx))
dict_struct = {k: [dic[k] for dic in list_struct if dic is not None] for k in list_struct[0]}
df = pd.DataFrame(dict_struct)
df

  1%|          | 32704/3378606 [00:11<03:47, 14696.78it/s]

Failed for molecule idx=2774781 SMILES=C


  3%|▎         | 98240/3378606 [00:15<03:01, 18043.55it/s]

Failed for molecule idx=3342150 SMILES=N
Failed for molecule idx=3347837 SMILES=O
Failed for molecule idx=3349913 SMILES=O
Failed for molecule idx=3348987 SMILES=P


  3%|▎         | 106432/3378606 [00:16<03:03, 17823.61it/s]

...

Failed for molecule idx=1838508 SMILES=N
Failed for molecule idx=1831559 SMILES=P


 97%|█████████▋| 3289024/3378606 [03:18<00:04, 18092.59it/s]

Failed for molecule idx=1841135 SMILES=S
Failed for molecule idx=1849751 SMILES=[NH]


100%|██████████| 3378606/3378606 [03:23<00:00, 16637.40it/s]


,I_a,I_b,I_c,IH_a,IH_b,IH_c,len_a,len_b,len_c,idx
0,1.801022,1.205763,0.703477,3.332717,2.230740,1.277720,0.785694,0.564729,0.180997,252506
1,0.848249,0.668207,0.180042,1.054542,0.906995,0.147547,0.781616,0.382377,0.000281,253520
2,1.723672,1.429719,0.451579,2.734154,2.413071,0.522670,0.860926,0.523788,0.239813,253300
3,1.479145,1.422294,0.184963,2.109702,2.044756,0.159232,0.976637,0.314264,0.220982,259104
4,0.799458,0.749364,0.148133,0.601163,0.524731,0.162983,0.673110,0.281897,0.213451,256366
...,...,...,...,...,...,...,...,...,...,...
3378521,2.039659,2.003402,0.355868,1.830906,1.700403,0.312575,1.006012,0.353471,0.361731,1175534
3378522,2.090909,1.976536,0.304126,4.400455,4.142253,0.713854,1.070125,0.503605,0.283449,1173024
3378523,1.033674,0.909330,0.271496,1.491752,1.324474,0.221520,0.769898,0.373006,0.267498,1177412
3378524,1.218129,1.101899,0.249071,1.317298,1.287553,0.284384,0.800118,0.361007,0.249172,1178625


In [9]:
raw_df = pd.read_csv(RAW_CSV)
raw_df

,idx,smiles,homolumogap
0,0,O=C1[N]c2ccncc2[CH][C@@H]1c1ccc(cc1)C,3.047675
1,1,COc1cc(OC)ccc1/C=C/N(C(=O)C)C,4.410966
2,2,C=CCN(C(=O)C)/C=C/c1ccccc1C,4.639541
3,3,C=CCN(C(=O)C)/C=C/c1ccccc1F,4.492600
4,4,C=CCN(C(=O)C)/C=C/c1ccccc1Cl,4.612330
...,...,...,...
3746615,3746615,CCn1cnc2c1ncnc2N,NaN
3746616,3746616,O=N(=O)c1ccc(c(c1)N(=O)=O)Cl,NaN
3746617,3746617,NCC(=O)COP(=O)(O)O,NaN
3746618,3746618,C[C@@H](CN)O,NaN


In [10]:
merged_df = raw_df.merge(df, how="inner", on=["idx"])
merged_df

,idx,smiles,homolumogap,I_a,I_b,I_c,IH_a,IH_b,IH_c,len_a,len_b,len_c
0,0,O=C1[N]c2ccncc2[CH][C@@H]1c1ccc(cc1)C,3.047675,1.813462,1.601682,0.401664,1.510950,1.436966,0.173702,0.961845,0.452937,0.254786
1,1,COc1cc(OC)ccc1/C=C/N(C(=O)C)C,4.410966,2.308566,1.932109,0.403518,2.688305,1.975483,0.773145,1.053264,0.561198,0.126349
2,2,C=CCN(C(=O)C)/C=C/c1ccccc1C,4.639541,1.691103,1.338676,0.398564,1.848389,1.220168,0.714471,0.869384,0.583278,0.161675
3,3,C=CCN(C(=O)C)/C=C/c1ccccc1F,4.492600,1.753497,1.367217,0.411020,1.552199,1.053055,0.540122,0.875695,0.588322,0.126418
4,4,C=CCN(C(=O)C)/C=C/c1ccccc1Cl,4.612330,1.872057,1.467153,0.527033,1.661891,1.365148,0.350127,0.891042,0.562013,0.210463
...,...,...,...,...,...,...,...,...,...,...,...,...
3378521,3378601,Cc1ccc(c(c1)C)N[C@H](/C(=N\C1CC1)/O)C,5.347037,1.348275,1.288381,0.529860,2.074233,1.926117,0.849037,0.828846,0.462148,0.401456
3378522,3378602,C[C@@H](/C(=N\C1CC1)/O)Nc1cccc(c1C)C,5.809631,1.667726,1.532551,0.408586,2.339997,1.954943,0.658524,0.886473,0.473849,0.327503
3378523,3378603,C[C@H](/C(=N\C(=N)O)/O)Nc1cccc(c1C)C,5.064039,1.450947,1.335399,0.385102,1.659290,1.385724,0.449171,0.919293,0.435652,0.331783
3378524,3378604,C[C@@H](/C(=N\C(=N)O)/O)Nc1cccc(c1C)C,5.336153,1.459332,1.360086,0.370713,1.663606,1.398784,0.437598,0.925686,0.434268,0.347544


In [11]:
merged_df = merged_df.sort_values(by="idx")
csv_path = os.path.join(ROOT, f"pcqm4m-v2-physics{micro_name}.csv.gz")
merged_df.to_csv(csv_path, index=False)